In [1]:
import sys
import os

sys.path.append("/home/laststar/source/open-llm-rec/source")

In [2]:
import easydict

In [3]:
# jupyter didn't support argparse. so, I use 'easydict' module
args = easydict.EasyDict({
    ################
    # Dataset
    ################
    'dataset_code': 'ml-100k', # ml-100k, beauty, games
    'min_rating': 0,  # default: 0
    'min_uc': 5,  # default: 5
    'min_sc': 5,  # default: 5
    'seed': 42,  # default: 42

    ################
    # Dataloader
    ################
    'train_batch_size': 64,  # default: 64
    'val_batch_size': 64,  # default: 64
    'test_batch_size': 64,  # default: 64
    'num_workers': 0,  # default: 8
    'sliding_window_size': 1.0,  # default: 1.0
    'negative_sample_size': 10,  # default: 10

    ################
    # Trainer
    ################
    # optimization #
    'device': 'cuda',  # default: 'cuda'  # choices: ['cpu', 'cuda']
    'num_epochs': 500,  # default: 500
    'optimizer': 'AdamW',  # default: 'AdamW'  # choices: ['AdamW', 'Adam']
    'weight_decay': 0.01,  # default: None
    'adam_epsilon': 1e-9,  # default: 1e-9
    'momentum': None,  # default: None
    'lr': 0.001,  # default: 0.001
    'max_grad_norm': 5.0,  # default: 5.0
    'enable_lr_schedule': True,  # default: True
    'decay_step': 10000,  # default: 10000
    'gamma': 1,  # default: 1
    'enable_lr_warmup': True,  # default: True
    'warmup_steps': 100,  # default: 100

    # evaluation #
    'val_strategy': 'iteration',  # default: 'iteration'  # choices: ['epoch', 'iteration']
    'val_iterations': 500,  # default: 500  # only for iteration val_strategy
    'early_stopping': True,  # default: True
    'early_stopping_patience': 20,  # default: 20
    'metric_ks': [1, 5, 10, 20, 50],  # default: [1, 5, 10, 20, 50]
    'rerank_metric_ks': [1, 5, 10],  # default: [1, 5, 10]
    'best_metric': 'Recall@10',  # default: 'Recall@10'
    'rerank_best_metric': 'NDCG@10',  # default: 'NDCG@10'
    'use_wandb': False,  # default: False

    ################
    # Retriever Model
    ################
    'model_code': 'lru',  # default: None
    'bert_max_len': 50,  # default: 50
    'bert_hidden_units': 64,  # default: 64
    'bert_num_blocks': 2,  # default: 2
    'bert_num_heads': 2,  # default: 2
    'bert_head_size': 32,  # default: 32
    'bert_dropout': 0.2,  # default: 0.2
    'bert_attn_dropout': 0.2,  # default: 0.2
    'bert_mask_prob': 0.25,  # default: 0.25

    ################
    # LLM Model
    ################
    'llm_base_model': 'meta-llama/Llama-2-7b-hf',  # default: 'meta-llama/Llama-2-7b-hf'
    'llm_base_tokenizer': 'meta-llama/Llama-2-7b-hf',  # default: 'meta-llama/Llama-2-7b-hf'
    'llm_max_title_len': 32,  # default: 32
    'llm_max_text_len': 1536,  # default: 1536
    'llm_max_history': 20,  # default: 20
    'llm_train_on_inputs': False,  # default: False
    'llm_negative_sample_size': 19,  # default: 19  # 19 negative & 1 positive
    'llm_system_template': "Given user history in chronological order, recommend an item from the candidate pool with its index letter.",  # default: "Given user history in chronological order, recommend an item from the candidate pool with its index letter."
    'llm_input_template': 'User history: {}; \n Candidate pool: {}',  # default: 'User history: {}; \n Candidate pool: {}'
    'llm_load_in_4bit': True,  # default: True
    'llm_retrieved_path': None,  # default: None
    'llm_cache_dir': None,  # default: None

    ################
    # Lora
    ################
    'lora_r': 8,  # default: 8
    'lora_alpha': 32,  # default: 32
    'lora_dropout': 0.05,  # default: 0.05
    'lora_target_modules': ['q_proj', 'v_proj'],  # default: ['q_proj', 'v_proj']
    'lora_num_epochs': 1,  # default: 1
    'lora_val_iterations': 100,  # default: 100
    'lora_early_stopping_patience': 20,  # default: 20
    'lora_lr': 1e-4,  # default: 1e-4
    'lora_micro_batch_size': 16,  # default: 16
})


# Dataset Loader

In [4]:
import data.datasets
import data.dataloader
from data.dataloader import *
from data.datasets import *

In [5]:
train, val, test = dataloader_factory(args)

Already preprocessed. Skip preprocessing


# Train

In [6]:
import os
import torch

os.environ['TOKENIZERS_PARALLELISM'] = 'false'

import wandb
import argparse

from config import *
from model import *
from data.dataloader import*
from trainer import *

In [7]:
export_root = "/home/laststar/data/model/open-llm-rec"

In [8]:
model = LRURec(args)

/home/laststar/anaconda3/envs/llama/lib/python3.11/site-packages/torch/nn/modules/module.py:1145: UserWarning: Complex modules are a new feature under active development whose design may change, and some modules might not work as expected when using complex tensors as parameters or buffers. Please file an issue at https://github.com/pytorch/pytorch/issues/new?template=bug-report.yml if a complex module does not work as expected.
  warnings.warn(


In [41]:
trainer = LRUTrainer(args, model, train, val, test, export_root, args.use_wandb)

{'dataset_code': 'ml-100k', 'min_rating': 0, 'min_uc': 5, 'min_sc': 5, 'seed': 42, 'train_batch_size': 64, 'val_batch_size': 64, 'test_batch_size': 64, 'num_workers': 0, 'sliding_window_size': 1.0, 'negative_sample_size': 10, 'device': 'cuda', 'num_epochs': 500, 'optimizer': 'AdamW', 'weight_decay': 0.01, 'adam_epsilon': 1e-09, 'momentum': None, 'lr': 0.001, 'max_grad_norm': 5.0, 'enable_lr_schedule': True, 'decay_step': 10000, 'gamma': 1, 'enable_lr_warmup': True, 'warmup_steps': 100, 'val_strategy': 'iteration', 'val_iterations': 500, 'early_stopping': True, 'early_stopping_patience': 20, 'metric_ks': [1, 5, 10, 20, 50], 'rerank_metric_ks': [1, 5, 10], 'best_metric': 'Recall@10', 'rerank_best_metric': 'NDCG@10', 'use_wandb': False, 'model_code': 'lru', 'bert_max_len': 50, 'bert_hidden_units': 64, 'bert_num_blocks': 2, 'bert_num_heads': 2, 'bert_head_size': 32, 'bert_dropout': 0.2, 'bert_attn_dropout': 0.2, 'bert_mask_prob': 0.25, 'llm_base_model': 'meta-llama/Llama-2-7b-hf', 'llm_bas

In [42]:
trainer.train()

Eval: N@1 0.0000, N@5 0.0000, N@10 0.0000, R@1 0.0000, R@5 0.0000, R@10 0.0000: 100%|██████████| 10/10 [00:00<00:00, 35.49it/s]
  0%|          | 0/10 [00:00<?, ?it/s]
Eval: N@1 0.0156, N@5 0.0217, N@10 0.0322, R@1 0.0156, R@5 0.0312, R@10 0.0625:   0%|          | 0/10 [00:00<?, ?it/s]
Eval: N@1 0.0078, N@5 0.0394, N@10 0.0495, R@1 0.0078, R@5 0.0781, R@10 0.1094:   0%|          | 0/10 [00:00<?, ?it/s]
Eval: N@1 0.0104, N@5 0.0440, N@10 0.0568, R@1 0.0104, R@5 0.0833, R@10 0.1250:   0%|          | 0/10 [00:00<?, ?it/s]
Eval: N@1 0.0104, N@5 0.0440, N@10 0.0568, R@1 0.0104, R@5 0.0833, R@10 0.1250:  30%|███       | 3/10 [00:00<00:00, 26.02it/s]
Eval: N@1 0.0078, N@5 0.0330, N@10 0.0475, R@1 0.0078, R@5 0.0625, R@10 0.1094:  30%|███       | 3/10 [00:00<00:00, 26.02it/s]
Eval: N@1 0.0063, N@5 0.0349, N@10 0.0486, R@1 0.0063, R@5 0.0688, R@10 0.1125:  30%|███       | 3/10 [00:00<00:00, 26.02it/s]
Eval: N@1 0.0052, N@5 0.0304, N@10 0.0461, R@1 0.0052, R@5 0.0599, R@10 0.1094:  30%|███       

Update Best Recall@10 Model at 19


  0%|          | 0/10 [00:00<?, ?it/s]
Eval: N@1 0.0000, N@5 0.0213, N@10 0.0361, R@1 0.0000, R@5 0.0469, R@10 0.0938:   0%|          | 0/10 [00:00<?, ?it/s]
Eval: N@1 0.0078, N@5 0.0389, N@10 0.0582, R@1 0.0078, R@5 0.0703, R@10 0.1328:   0%|          | 0/10 [00:00<?, ?it/s]
Eval: N@1 0.0052, N@5 0.0361, N@10 0.0554, R@1 0.0052, R@5 0.0677, R@10 0.1302:   0%|          | 0/10 [00:00<?, ?it/s]
Eval: N@1 0.0052, N@5 0.0361, N@10 0.0554, R@1 0.0052, R@5 0.0677, R@10 0.1302:  30%|███       | 3/10 [00:00<00:00, 27.07it/s]
Eval: N@1 0.0039, N@5 0.0325, N@10 0.0494, R@1 0.0039, R@5 0.0625, R@10 0.1172:  30%|███       | 3/10 [00:00<00:00, 27.07it/s]
Eval: N@1 0.0063, N@5 0.0311, N@10 0.0475, R@1 0.0063, R@5 0.0563, R@10 0.1094:  30%|███       | 3/10 [00:00<00:00, 27.07it/s]
Eval: N@1 0.0052, N@5 0.0302, N@10 0.0439, R@1 0.0052, R@5 0.0547, R@10 0.0990:  30%|███       | 3/10 [00:00<00:00, 27.07it/s]
Eval: N@1 0.0067, N@5 0.0339, N@10 0.0456, R@1 0.0067, R@5 0.0603, R@10 0.0982:  30%|███       |

Early stopping triggered. Exit training


In [43]:
from datetime import datetime
print("current time:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

current time: 2024-10-05 16:34:38


In [44]:
# 10/5 4:09 train start 600MB used

In [46]:
from datetime import datetime
print("current time:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

current time: 2024-10-05 16:48:27


In [49]:
trainer.test()

/home/laststar/source/open-llm-rec/source/trainer/base.py:127: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  best_model_dict = torch.load(os.path.join(


******************** Testing Best Model ********************


Eval: N@1 0.0217, N@5 0.0459, N@10 0.0621, R@1 0.0217, R@5 0.0731, R@10 0.1225: 100%|██████████| 10/10 [00:00<00:00, 27.74it/s]

******************** Testing Metrics ********************
{'Recall@50': 0.2825367659330368, 'MRR@50': 0.0507874995470047, 'NDCG@50': 0.09637335240840912, 'Recall@20': 0.17545955926179885, 'MRR@20': 0.04745325967669487, 'NDCG@20': 0.07523392103612422, 'Recall@10': 0.1225183829665184, 'MRR@10': 0.043945787474513054, 'NDCG@10': 0.0620549526065588, 'Recall@5': 0.07306985296308995, 'MRR@5': 0.0372012882027775, 'NDCG@5': 0.04591377899050712, 'Recall@1': 0.02169117648154497, 'MRR@1': 0.02169117648154497, 'NDCG@1': 0.02169117648154497}


{'Recall@50': 0.2825367659330368,
 'MRR@50': 0.0507874995470047,
 'NDCG@50': 0.09637335240840912,
 'Recall@20': 0.17545955926179885,
 'MRR@20': 0.04745325967669487,
 'NDCG@20': 0.07523392103612422,
 'Recall@10': 0.1225183829665184,
 'MRR@10': 0.043945787474513054,
 'NDCG@10': 0.0620549526065588,
 'Recall@5': 0.07306985296308995,
 'MRR@5': 0.0372012882027775,
 'NDCG@5': 0.04591377899050712,
 'Recall@1': 0.02169117648154497,
 'MRR@1': 0.02169117648154497,
 'NDCG@1': 0.02169117648154497}

In [48]:
from datetime import datetime
print("current time:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

current time: 2024-10-05 16:48:28


In [50]:
trainer.generate_candidates(os.path.join(export_root, 'retrieved.pkl'))

*************** Generating Candidates for Validation Set ***************


100%|██████████| 10/10 [00:00<00:00, 45.31it/s]


{'Recall@50': 0.3049180209636688, 'MRR@50': 0.06256759166717529, 'NDCG@50': 0.11019700765609741, 'Recall@20': 0.18852458894252777, 'MRR@20': 0.05895635113120079, 'NDCG@20': 0.08729086071252823, 'Recall@10': 0.12950819730758667, 'MRR@10': 0.05494340509176254, 'NDCG@10': 0.07247135043144226, 'Recall@5': 0.08852458745241165, 'MRR@5': 0.04918033257126808, 'NDCG@5': 0.05892314016819, 'Recall@1': 0.027868852019309998, 'MRR@1': 0.027868852019309998, 'NDCG@1': 0.027868852019309998}
****************** Generating Candidates for Test Set ******************


100%|██████████| 10/10 [00:00<00:00, 53.49it/s]


{'Recall@50': 0.2819672226905823, 'MRR@50': 0.05073552951216698, 'NDCG@50': 0.09626712650060654, 'Recall@20': 0.17540983855724335, 'MRR@20': 0.04737967252731323, 'NDCG@20': 0.0751628652215004, 'Recall@10': 0.1213114783167839, 'MRR@10': 0.04380302131175995, 'NDCG@10': 0.06170599162578583, 'Recall@5': 0.07377049326896667, 'MRR@5': 0.037295080721378326, 'NDCG@5': 0.04616577923297882, 'Recall@1': 0.021311474964022636, 'MRR@1': 0.021311474964022636, 'NDCG@1': 0.021311474964022636}
